DSPC: Creating complex molecules using mBuild's hierarchical design
===================================

__Note__: This tutorial assumes completion of the Methane, Ethane, and Polymer tutorials, and a mastery of the basics concepts of mBuild (`Compounds`, `Ports`, basic coordinate transforms)

Let's begin by importing some useful molecular units to build our DSPC molecule, as well as the mBuild package of course:

In [56]:
import mbuild as mb

from mbuild.prototypes import AlkylMonomer, COOH
from mbuild.lib.moieties import CH3

Now, we could try to build a DSPC molecule directly from these small molecular components.

However, this would be cumbersome and would fail to take advantage of mBuild's intelligent design (not to mention, it would fly in the face of Object-Oriented Programming as a whole. _Sad_!)

Therefore, let's start with a smaller unit that can be reused later, like a *free fatty acid*. 

### Free Fatty Acid Construction
Here, we initialize the `Compound` and add a `COOH` headgroup.

In [57]:
ffa = mb.Compound()

ffa.add(COOH(ester=True), label='head')

Notice the 'ester=True' keyword parameter. 

This is included because, in the future, we will likely want to connect this free fatty acid to another `Compound`, so we give the headgroup ester behavior to accomodate this future connection.

Now, onto the tail (utilizing the `Polymer` class) and tailcap.

In [58]:
ffa.add(mb.Polymer(AlkylMonomer(), 18), label='tail')
ffa.add(CH3(), label='tailcap')

Finally, let's connect these pieces and see what happens.

In [59]:
mb.force_overlap(move_this=ffa['tailcap'], from_positions=ffa['tailcap']['up'],
                 to_positions=ffa['tail']['up'])
mb.force_overlap(move_this=ffa['head'], from_positions=ffa['head']['up'],
                 to_positions=ffa['tail']['down'])

ffa.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Well that's odd. Our `Ports` are all connected correctly, but the head group isn't displaying normal geometry.

This happens a lot in mBuild when we create bonds with Ports. We can specify orientation and distance correctly, but because bonds are rotatable, there is no guarantee that the two Compounds will be rotated correctly with respect to each other.

To fix this, we'll have to spin the `Port` connected to the headgroup in-place by 90 degrees. Insert this line of code into the cell below, right after the line where the headgroup is added. __Note__: mBuild's rotational units are in radians, so we'll need to make use of the NumPy package.

```
ffa['head']['up'].spin(np.pi/2, ffa['head']['C'].pos)
```



In [60]:
import numpy as np
import mbuild as mb

from mbuild.prototypes import *
from mbuild.lib.moieties import CH3

ffa = mb.Compound()

ffa.add(COOH(ester=True), label='head')

ffa.add(mb.Polymer(AlkylMonomer(), 18), label='tail')
ffa.add(CH3(), label='tailcap')
mb.force_overlap(move_this=ffa['tailcap'], from_positions=ffa['tailcap']['up'],
                 to_positions=ffa['tail']['up'])
mb.force_overlap(move_this=ffa['head'], from_positions=ffa['head']['up'],
                 to_positions=ffa['tail']['down'])

ffa.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


For proper integration with GROMACS simulation file types, some of the names of the atoms have been changed, which is why one of the oxygen atoms is colored incorrectly. Rest assured, your free fatty acid has been constructed correctly if you've followed the steps above.

Now, in order to make this component reusable, we need to wrap it in a Python class (if you want to see the way the molecule looks without the ester property, simply specify `ester=False`).

___Tip___: Your programs will only be as useful as they are flexible. Notice that the class below incorporates a `chain_length` parameter, allowing for the creation of many lengths of free fatty acids.

In [61]:
import mbuild as mb
import numpy as np

from mbuild.prototypes import *
from mbuild.lib.moieties.ch3 import CH3

class FFA(mb.Compound):
    """Creates a saturated free fatty acid of n carbons based on user
    input for the united atom model"""
    def __init__(self, chain_length, ester=True):
        super(FFA, self).__init__()
        
        if ester:
            self.add(COOH(ester=True), label='head')
        
        tail = mb.Polymer(AlkylMonomer(), (chain_length - 2))
        if not ester:
            self.add(CH3(), label='tailcap')
        self.add(tail, label='tail')
        if ester:
            self.add(CH3(), label='tailcap')
        mb.x_axis_transform(self['tail'], new_origin=self['tail'][0],
                point_on_x_axis=self['tail'][2],
                point_on_xy_plane=self['tail'][1])

        if not ester:
            self.add(COOH(), label='head')
        if ester:
            mb.force_overlap(move_this=self['tailcap'],
                from_positions=self['tailcap']['up'],
                to_positions=self['tail']['up'])
        else:
            mb.force_overlap(move_this=self['tailcap'],
                from_positions=self['tailcap']['up'],
                to_positions=self['tail']['down'])

        self['head']['up'].spin(-np.pi/2, self['head']['up'].pos)
        if ester: 
            mb.force_overlap(move_this=self['head'],
                from_positions=self['head']['up'],
                to_positions=self['tail']['down'])
            self.spin(np.pi/2, [0,1,0])
        else:
            mb.force_overlap(move_this=self['head'],
                from_positions=self['head']['up'],
                to_positions=self['tail']['up'])
            self.spin(-np.pi/2, [0,1,0])
        self.name = 'FFA' + str(chain_length)

In [62]:
ffa = FFA(18, ester=False)
ffa.visualize()


/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Now that we've demonstrated how you can build a molecule like a free fatty acid from the ground up, we'll spend the rest of the tutorial connecting previously created molecular components to make larger molecules.

### Phosphatidyl choline tail group construction

The tails of a DSPC molecule are simply free fatty acids connected in a glycerol, which makes constructing them very easy, since we stuck to mBuild's hierarchical design! A visual of the tail group can be found [here](https://en.wikipedia.org/wiki/Diglyceride#/media/File:1,2-diacylglycerol.svg).

Let's start by initializing and adding the 2-carbon chain:

In [63]:
pctails = mb.Compound()
pctails.add(mb.Polymer(AlkylMonomer(), 2), label='base')

pctails.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


If you looked closely at the picture of the tail group construction, you noticed that there was an _O-H_ branching off one of the carbons in the tail groups.

This is where the head group is going to go, so we need to replace a hydrogen `Particle` on one of these carbons with a `Port`.

To do this, we will place the `Port` where one of the hydrogens is currently located. After placing the `Port`, we remove the hydrogen `Particle`.

In [66]:
pctails.translate(-pctails['base'][3].pos)
pctails['base'].add(mb.Port(anchor=pctails['base'][3],
                            orientation=pctails['base'][5].pos, 
                            separation=.143/2), label='side')
pctails.remove(pctails['base'][5])

pctails.visualize(show_ports=True)

IndexError: list index out of range

Now, let's connect our free fatty acid tails to our base.

In [65]:
pctails.add(FFA(18, ester=True), label='FFA[$]')
mb.force_overlap(move_this=pctails['base'], 
                 from_positions=pctails['base']['down'],
                 to_positions=pctails['FFA'][0]['head']['down'])
pctails.add(FFA(18, ester=True), label='FFA[$]')
mb.force_overlap(move_this=pctails['FFA'][1], 
                 from_positions=pctails['FFA'][1]['head']['down'],
                 to_positions=pctails['base']['side'])

pctails.visualize()
        


/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Unsurprisingly, there are some `Ports` on either the base or the tails that need to be rotated due to strange geometry. Rotating the `Ports` on the base is easier to visualize, so insert the following lines to the cell where we created the base, just after removing the hydrogen. ___Bug:___ _Due to how we imported some things, you may have to execute all cells again starting at tail group creation._

```
pctails['base']['side'].spin(np.pi, pctails['base']['side'].pos)
pctails.translate(-pctails['base'][0].pos)
pctails['base']['down'].spin(np.pi/2, pctails['base']['down'].pos)
```

Now that we've created the tails for the DSPC molecule, let's make sure to wrap them in a Python class to make them reusable.

__Note__: Once again, our class has flexibility because of the `tail_i_length` arguments. We will use two lengths of 18 for a DSPC molecule.

In [95]:
class PCTails(mb.Compound):
    def __init__(self, tail_1_length, tail_2_length):
        super(PCTails, self).__init__()
        
        self.add(mb.Polymer(AlkylMonomer(), 2), label='base')
        self.translate(-self['base'][3].pos)
        self['base'].add(mb.Port(anchor=self['base'][3],
            orientation=self['base'][5].pos, 
            separation=.143/2), label='side')
        self.remove(self['base'][5])
        self['base']['side'].spin(np.pi, self['base']['side'].pos)
        self.translate(-self['base'][0].pos)
        self['base']['down'].spin(np.pi/2, self['base']['down'].pos)
        
        self.add(FFA(tail_1_length, ester=True), label='FFA[$]')
        mb.force_overlap(move_this=self['base'], 
                from_positions=self['base']['down'],
                to_positions=self['FFA'][0]['head']['down'])
        self.add(FFA(tail_2_length, ester=True), label='FFA[$]')
        mb.force_overlap(move_this=self['FFA'][1], 
                from_positions=self['FFA'][1]['head']['down'],
                to_positions=self['base']['side'])

In [96]:
pctails = PCTails(18, 18)
pctails.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Now that the tail group has been constructed, you might guess what the final step is.

### Connecting the head group

Let's import the head group (which was created using the skills you've hopefully learned by doing the mBuild tutorials), and attach it to the tail group.

In [97]:
from mbuild.prototypes import PCHead

dspc = mb.Compound()
dspc.add(PCHead(), label='headgroup')
dspc.add(PCTails(18,18), label='ffatails')
        
mb.force_overlap(move_this=dspc['ffatails'],
                 from_positions=dspc['ffatails']['base']['up'],
                 to_positions=dspc['headgroup']['alkyl_split']['up'])

dspc.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1241: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Congratulations! You're looking at a DSPC molecule constructed 100% in mBuild. In addition, we've utilized OOP so well, that if one wanted to make another phosphatidylcholine molecule (such as DPPC), all one has to do is change the tail lengths!

In [98]:
dppc = mb.Compound()
dppc.add(PCHead(), label='headgroup')
dppc.add(PCTails(16,16), label='ffatails')
        
mb.force_overlap(move_this=dppc['ffatails'],
                 from_positions=dppc['ffatails']['base']['up'],
                 to_positions=dppc['headgroup']['alkyl_split']['up'])

dppc.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1241: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


As always, we'll finish off by wrapping our DSPC molecule in a Python class. For a cool way to use this molecule, see the Bilayer Builder tutorial! Happy molecule construction!

In [99]:
import mbuild as mb
import numpy as np


from mbuild.prototypes import PCHead

class DSPC(mb.Compound):
    def __init__(self):
        super(DSPC, self).__init__()
        
        self.add(PCHead(), label='headgroup')
        self.add(PCTails(18,18), label='ffatails')
        
        mb.force_overlap(move_this=self['ffatails'],
                        from_positions=self['ffatails']['base']['up'],
                        to_positions=self['headgroup']['alkyl_split']['up'])
    

In [100]:
dspc = DSPC()
dspc.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1241: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.
